In [ ]:
import numpy as np
from airo_drake import DualArmScene, add_floor, add_manipulator, add_meshcat, add_wall, finish_build
from pydrake.math import RigidTransform, RollPitchYaw
from pydrake.multibody.tree import ModelInstanceIndex
from pydrake.planning import RobotDiagramBuilder, SceneGraphCollisionChecker
from cloth_tools.drake.scenes import add_cloth_competition_dual_ur5e_scene, X_W_LCB_DEFAULT, X_W_RCB_DEFAULT
from airo_drake import X_URBASE_ROSBASE

robot_diagram_builder = RobotDiagramBuilder()

meshcat = add_meshcat(robot_diagram_builder)
meshcat.SetCameraPose([-1.5, 0, 1.0], [0, 0, 0])

X_W_LCB = X_W_LCB_DEFAULT
X_W_RCB = X_W_RCB_DEFAULT

(arm_left_index, arm_right_index), (gripper_left_index, gripper_right_index) = add_cloth_competition_dual_ur5e_scene(
    robot_diagram_builder, X_W_LCB, X_W_RCB
)
robot_diagram, context = finish_build(robot_diagram_builder, meshcat)

scene = DualArmScene(robot_diagram, arm_left_index, arm_right_index, gripper_left_index, gripper_right_index, meshcat)

In [ ]:
collision_checker = SceneGraphCollisionChecker(
    model=scene.robot_diagram,
    robot_model_instances=[
        scene.arm_left_index,
        scene.arm_right_index,
        scene.gripper_left_index,
        scene.gripper_right_index,
    ],
    edge_step_size=0.125,  # Arbitrary value: we don't use the CheckEdgeCollisionFree
    env_collision_padding=0.005,
    self_collision_padding=0.005,
)

In [ ]:
home_joints_left = np.deg2rad([180, -120, 60, -30, -90, -90])
home_joints_right = np.deg2rad([-180, -60, -60, -150, 90, 90])
home_joints = np.concatenate([home_joints_left, home_joints_right])

plant = scene.robot_diagram.plant()
plant_context = plant.GetMyContextFromRoot(context)
plant.SetPositions(plant_context, home_joints)
scene.robot_diagram.ForcedPublish(context) # updates the meshcat visualization


In [ ]:
from functools import partial
from cloth_tools.kinematics.constants import TCP_TRANSFORM
from cloth_tools.kinematics.inverse_kinematics import inverse_kinematics_in_world_fn

inverse_kinematics_left_fn = partial(inverse_kinematics_in_world_fn, X_W_CB=X_W_LCB, tcp_transform=TCP_TRANSFORM)
inverse_kinematics_right_fn = partial(inverse_kinematics_in_world_fn, X_W_CB=X_W_RCB, tcp_transform=TCP_TRANSFORM)

In [ ]:
from airo_drake import visualize_frame
from cloth_tools.planning.grasp_planning import make_lowest_grasp_path_candidates

lowest_point = np.array([0.2, 0.0, 0.2])
grasp_depth = 0.0

grasp_pose_paths = make_lowest_grasp_path_candidates(lowest_point, grasp_depth)
print(len(grasp_pose_paths))

In [ ]:
for i, grasp_pose_path in enumerate(grasp_pose_paths):
    pregrasp_pose = grasp_pose_path[0]
    visualize_frame(scene.meshcat, f"pregrasp_pose/pose_{i}", pregrasp_pose, length=0.05)

In [ ]:
from airo_planner import DualArmOmplPlanner

joint_bounds_lower = np.deg2rad([-360, -195, -160, -360, -360, -360])
joint_bounds_upper = np.deg2rad([360, 15, 160, 360, 360, 360])
joint_bounds = joint_bounds_lower, joint_bounds_upper

planner = DualArmOmplPlanner(
    is_state_valid_fn=collision_checker.CheckConfigCollisionFree,
    inverse_kinematics_left_fn=inverse_kinematics_left_fn,
    inverse_kinematics_right_fn=inverse_kinematics_right_fn,
    joint_bounds_left=joint_bounds,
    joint_bounds_right=joint_bounds,
)

In [ ]:
from cloth_tools.planning.grasp_planning import GraspNotFeasibleError, plan_to_grasp_pose_path
from loguru import logger

grasp_pose_path = grasp_pose_paths[55]
# grasp_pose_path = grasp_pose_paths[35]

for i, tcp_pose in enumerate(grasp_pose_path):
    visualize_frame(scene.meshcat, f"tcp_poses/pose_{i}", tcp_pose, length=0.05)

trajectory = None
try:
    trajectory = plan_to_grasp_pose_path(
        planner,
        grasp_pose_path,
        home_joints_left,
        home_joints_right,
        inverse_kinematics_left_fn,
        inverse_kinematics_right_fn,
        collision_checker.CheckConfigCollisionFree,
        plant,
        with_left=True
    )
    logger.success("Grasp feasible")
except GraspNotFeasibleError:
    logger.error("Grasp not feasible")

In [ ]:
meshcat.Delete("tcp_poses/")

In [ ]:
from airo_drake import animate_dual_joint_trajectory

animate_dual_joint_trajectory(scene.meshcat, scene.robot_diagram, scene.arm_left_index, scene.arm_right_index, trajectory)

In [ ]:
from cloth_tools.planning.grasp_planning import ExhaustedOptionsError, GraspNotFeasibleError, plan_lowest_point_grasp
from loguru import logger

trajectory = plan_lowest_point_grasp(
    lowest_point,
    grasp_depth,
    planner,
    home_joints_left,
    home_joints_right,
    inverse_kinematics_left_fn,
    inverse_kinematics_right_fn,
    collision_checker.CheckConfigCollisionFree,
    plant,
)

In [ ]:
from airo_drake import animate_dual_joint_trajectory

animate_dual_joint_trajectory(scene.meshcat, scene.robot_diagram, scene.arm_left_index, scene.arm_right_index, trajectory)

In [ ]:
scene.meshcat.Delete(f"pregrasp_pose/")